In [1]:
def CopyClientData(lakeName, instance: Enum):
  print("LakeName:", lakeName)
  print("Instance:", instance.name)
  print("InstanceId:", instance.value)
  
  # get last date from the loaded dataset
  LastSyncDate = GetLastSyncDate('geofences', 'client.'+instance.name)
  
  print("Reading data since", LastSyncDate)  
  print("Load Geofences data")
  PrepareDataBronze (
    lakeName = lakeName,
    instance = instance, 
    entity = BronzeTable.Geofences,
    entityName = 'tmpGeofences',
    startDate = LastSyncDate
  )
  
  print("Load GeofenceTypes data")
  PrepareDataBronze (
    lakeName = lakeName,
    instance = instance, 
    entity = BronzeTable.GeofenceTypes,
    entityName = 'tmpGeofenceTypes',
    startDate = LastSyncDate
  )
  
  print("Writing data...")
  
  sourceData = spark.sql("""
    select
          """+str(instance.value)+""" as InstanceId
        , gf.CompanyId 
        , gf.GeofenceId 
        , gf.Name 
        , gf.GeofenceTypeId 
        , coalesce(gft.Name, 'BAD DATA') as GeofenceType 
        , gf.ShapeType 
        , if(gf.ShapeType is null, null, if(sht.ShapeTypeId is null, 'BAD DATA', sht.ShapeTypeName)) as ShapeTypeString 
        , gf.Latitude 
        , gf.Longitude 
        , gf.RadiusCm 
        , gf.City 
        , gf.RegionName 
        , gf.PostalCode 
        , gf.AddressLine1 
        , gf.AddressLine2 
        , gf.Points 
        , gf.CountryCode 
        , gf.Deleted 
        , gf.CreatedUtc 
        , gf.ModifiedUtc 
        , gf.MBRMinLatitude 
        , gf.MBRMinLongitude 
        , gf.MBRMaxLatitude 
        , gf.MBRMaxLongitude 
        , gf.SubRegionName 
        , gf.RegionCode 
        , gf.SubRegionCode 
        , gf.UseGeofenceAddress 
        , current_timestamp() as SilverModifiedUtc
    from tmpGeofences gf
      left join tmpGeofenceTypes gft on gft.GeofenceTypeId = gf.GeofenceTypeId and gft.CompanyId = gf.CompanyId
      left join tmpShapeTypes sht on sht.ShapeTypeId = gf.ShapeType 
  """)  
  
  sourceData.registerTempTable("SourceData")
  
  print('...') 
  spark.sql("""
    merge into silver.Geofences as t
    using SourceData as s
      on t.GeofenceId = s.GeofenceId and t.CompanyId = s.CompanyId and t.InstanceId = s.InstanceId 
    when matched then update set 
              Name = s.Name 
            , GeofenceTypeId = s.GeofenceTypeId 
            , GeofenceType = s.GeofenceType 
            , ShapeType = s.ShapeType 
            , ShapeTypeString = s.ShapeTypeString 
            , Latitude = s.Latitude 
            , Longitude = s.Longitude 
            , RadiusCm = s.RadiusCm 
            , City = s.City 
            , RegionName = s.RegionName 
            , PostalCode = s.PostalCode 
            , AddressLine1 = s.AddressLine1 
            , AddressLine2 = s.AddressLine2 
            , Points = s.Points 
            , CountryCode = s.CountryCode 
            , Deleted = s.Deleted 
            , CreatedUtc = s.CreatedUtc 
            , ModifiedUtc = s.ModifiedUtc 
            , MBRMinLatitude = s.MBRMinLatitude 
            , MBRMinLongitude = s.MBRMinLongitude 
            , MBRMaxLatitude = s.MBRMaxLatitude 
            , MBRMaxLongitude = s.MBRMaxLongitude 
            , SubRegionName = s.SubRegionName 
            , RegionCode = s.RegionCode 
            , SubRegionCode = s.SubRegionCode 
            , UseGeofenceAddress = s.UseGeofenceAddress 
            , SilverModifiedUtc = s.SilverModifiedUtc
    when not matched then 
      insert ( InstanceId, CompanyId, GeofenceId, Name, GeofenceTypeId, GeofenceType, ShapeType, ShapeTypeString, Latitude, Longitude, RadiusCm, City, RegionName, PostalCode
             , AddressLine1, AddressLine2, Points, CountryCode, Deleted, CreatedUtc, ModifiedUtc, MBRMinLatitude, MBRMinLongitude, MBRMaxLatitude, MBRMaxLongitude, SubRegionName
             , RegionCode, SubRegionCode, UseGeofenceAddress, SilverModifiedUtc )
      values ( InstanceId, CompanyId, GeofenceId, Name, GeofenceTypeId, GeofenceType, ShapeType, ShapeTypeString, Latitude, Longitude, RadiusCm, City, RegionName, PostalCode
             , AddressLine1, AddressLine2, Points, CountryCode, Deleted, CreatedUtc, ModifiedUtc, MBRMinLatitude, MBRMinLongitude, MBRMaxLatitude, MBRMaxLongitude, SubRegionName
             , RegionCode, SubRegionCode, UseGeofenceAddress, SilverModifiedUtc )
  """);

  print("Writing data completed.")
  
  LastSyncDate = sourceData.agg({"ModifiedUtc": "max"}).collect()[0][0]
  print(LastSyncDate)

  if LastSyncDate is not None:
    UpdateLastSyncDate(LastSyncDate, 'geofences', 'client.'+instance.name)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1814531140542685> in <module> () 
 ----> 1 def CopyClientData ( lakeName , instance : Instance ) : 
 2 print ( lakeName ) 
 3 print ( "Instance:" , instance . name ) 
 4 print ( "InstanceId:" , instance . value ) 
 5 

 NameError : name 'Instance' is not defined